In [1]:
# Imports
import pandas as pd
import lib.SAAT as SAAT
import lib.utils as utils

In [2]:
# Read & Clean
data = pd.read_csv("data/example_data.csv")

# Get department from email
data["Department"] = data["Email"].map(lambda email: utils.email_to_department(email))
data = data[data['Topic'] == 'Topic1']

data.head()

,Name,Email,Years_In_Org,Grade,Setup,Topic,1-2pm,2-3pm,3-4pm,4-5pm,...,2-3pm2,3-4pm2,4-5pm2,5-6pm2,1-2pm.1,2-3pm3,3-4pm3,4-5pm3,5-6pm3,Department
0,Person1,Person1@DepartmentG.com,<1 year,Level1,0,Topic1,0,0,1,1,...,0,1,1,1,0,0,1,1,1,departmentg
9,Person10,Person10@DepartmentF.com,1-3 years,Level2,0,Topic1,1,1,0,0,...,0,0,0,0,0,1,1,0,0,departmentf
11,Person12,Person12@DepartmentH.com,5-10 years,Level2,1,Topic1,0,0,0,1,...,1,1,1,1,1,1,1,1,0,departmenth
15,Person16,Person16@DepartmentL.com,1-3 years,Level2,0,Topic1,0,0,0,1,...,0,0,1,1,0,0,0,0,0,departmentl
16,Person17,Person17@DepartmentG.com,>10 years,Level2,0,Topic1,1,1,1,0,...,1,1,0,0,0,1,0,1,0,departmentg


In [3]:
# Initialise SAAT
my_SAAT = SAAT.SAAT(
    times = data.loc[:,'1-2pm':'5-6pm3'], 
    group_size = 4, 
    organise = data["Setup"], 
    department = data["Department"], 
    time_in_company = data["Years_In_Org"])

my_SAAT.set_schedule(my_SAAT.auto(minutes=0.2))
my_SAAT.copy_strategy = "slice" # since our state is just a list, slice is the fastest way to copy

#Run
state, e = my_SAAT.anneal()

/home/jovyan/SAAT/lib/SAAT.py:92: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if ("Yes" in y):
/home/jovyan/SAAT/lib/SAAT.py:94: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif ("If needed" in y):
 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00670          5.17    20.95%     0.00%     0:00:30    -1:59:57 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00670          5.22    17.27%     0.00%     0:00:13     0:00:00

In [37]:
# Print results
print('Allocation: ' + str(state))
print('Energy minimised at: ' + str(e))
utils.print_pretty_allocations(
    state = state, 
    times = data.loc[:,'1-2pm':'5-6pm3'], 
    group_size = 4,
    extras = data.iloc[:,2:6])

Allocation: [4, 8, 1, 11, 22, 16, 10, 9, 6, 23, 14, 15, 18, 17, 13, 20, 5, 7, 12, 21, 2, 19, 3]
Energy minimised at: 5.1517150742760505

3  0 0 0 1 1 0 0 0 1 1 0 0 0 0 0 ---- 1-3 years ---- Level2 ---- 0 ---- Topic1
7  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ---- 5-10 years ---- Level3 ---- 1 ---- Topic1
0  0 0 1 1 1 0 0 1 1 1 0 0 1 1 1 ---- <1 year ---- Level1 ---- 0 ---- Topic1
10 0 0 0 1 1 0 0 0 1 1 0 0 0 0 1 ---- >10 years ---- Level2 ---- 1 ---- Topic1

21 1 1 1 0 0 1 1 1 0 0 1 1 1 0 0 ---- 3-5 years ---- Level2 ---- 0 ---- Topic1
15 1 1 1 1 0 1 1 1 1 0 1 1 1 1 0 ---- <1 year ---- Level1 ---- 0 ---- Topic1
9  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ---- >10 years ---- Level3 ---- 0 ---- Topic1
8  1 1 0 0 0 1 1 1 1 0 1 1 1 0 0 ---- 1-3 years ---- Level2 ---- 1 ---- Topic1

5  1 1 0 0 1 1 0 0 1 1 1 1 1 1 1 ---- >10 years ---- Level3 ---- 1 ---- Topic1
22 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 ---- 1-3 years ---- Level2 ---- 0 ---- Topic1
13 1 0 1 1 0 1 0 1 1 0 1 0 1 0 0 ---- 1-3 years ---- Level2 ---- 0 ---- To